# ChatBot for CodeClause By - Aditya Punia

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

### Loading Data

In [4]:
data_file = 'intents.json'
with open(data_file, "r", encoding="utf-8") as json_file:
    intents = json.load(json_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [9]:
classes = sorted(list(set(classes)))

In [10]:
print (len(documents), "documents")

50 documents


In [11]:
print (len(classes), "classes", classes)

10 classes ['About CodeClause', 'Contact', 'goodbye', 'greeting', 'internships', 'internships duration', 'internships fields', 'offer_letter', 'options', 'thanks']


In [12]:
print (len(words), "unique lemmatized words", words)

75 unique lemmatized words ["'s", ',', 'about', 'an', 'anyone', 'applied', 'apply', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'codeclause', 'concern', 'contact', 'could', 'day', 'do', 'duration', 'field', 'for', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'in', 'inernship', 'information', 'internship', 'is', 'later', 'letter', 'long', 'many', 'me', 'more', 'my', 'next', 'nice', 'not', 'of', 'offer', 'offered', 'provide', 'receieve', 'recieved', 'see', 'service', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'touch', 'what', 'when', 'where', 'which', 'will', 'with', 'you']


In [13]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

### Training the ChatBot

In [14]:
# creating our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

In [15]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [16]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

C:\Users\adity\AppData\Local\Temp\ipykernel_1436\3232993347.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [17]:
# create train and test lists. X - patterns, Y - intents

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [18]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [19]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Users\adity\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
#fitting and saving the model 

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
10/10 [==============================] - 2s 8ms/step - loss: 2.3538 - accuracy: 0.1200
Epoch 2/200
10/10 [==============================] - 0s 9ms/step - loss: 2.2351 - accuracy: 0.2400
Epoch 3/200
10/10 [==============================] - 0s 6ms/step - loss: 2.1992 - accuracy: 0.1800
Epoch 4/200
10/10 [==============================] - 0s 7ms/step - loss: 2.0810 - accuracy: 0.3000
Epoch 5/200
10/10 [==============================] - 0s 7ms/step - loss: 1.9300 - accuracy: 0.3800
Epoch 6/200
10/10 [==============================] - 0s 6ms/step - loss: 1.8728 - accuracy: 0.3600
Epoch 7/200
10/10 [==============================] - 0s 6ms/step - loss: 1.7118 - accuracy: 0.4400
Epoch 8/200
10/10 [==============================] - 0s 8ms/step - loss: 1.6567 - accuracy: 0.4400
Epoch 9/200
10/10 [==============================] - 0s 8ms/step - loss: 1.6039 - accuracy: 0.5000
Epoch 10/200
10/10 [==============================] - 0s 10ms/step - loss: 1.3184 - accuracy: 0.6400
Epoch 11

Epoch 83/200
10/10 [==============================] - 0s 5ms/step - loss: 0.1576 - accuracy: 0.9200
Epoch 84/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0696 - accuracy: 0.9800
Epoch 85/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0736 - accuracy: 0.9800
Epoch 86/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0888 - accuracy: 0.9800
Epoch 87/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0804 - accuracy: 0.9800
Epoch 88/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0278 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0701 - accuracy: 0.9800
Epoch 90/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0481 - accuracy: 0.9800
Epoch 91/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0947 - accuracy: 0.9800
Epoch 92/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0964 - accuracy: 0.9800


10/10 [==============================] - 0s 7ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0367 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 167/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0797 - accuracy: 0.9600
Epoch 168/200
10/10 [==============================] - 0s 7ms/step - loss: 0.0237 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0549 - accuracy: 0.9800
Epoch 170/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0179 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0185 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0214 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 6ms/step - loss: 0.0691 - accuracy: 0.9800
Epoc

### Making Model ready to Chat

In [21]:
from keras.models import load_model

In [22]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json', encoding="utf-8").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

#### Creating Functions

In [23]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    
    return sentence_words

In [24]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [25]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [26]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [27]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [28]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [34]:
def start_chat():
    name = input("Enter Your Name: ")
    print("Welcome " + name + " to the Code Clause Bot Service! Let me know how can I help you?\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end" or "bye" in inp.lower():
            print("Code Clause : Bye! Come back again soon and have a nice day.")
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Code Clause : {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [33]:
# start the chat bot
start_chat()

Enter Your Name: Aditya Punia
Welcome Aditya Punia to the Code Clause Bot Service! Let me know how can I help you?

What all can you do
1/1 [==============================] - 0s 325ms/step
Code Clause: I can give you info about the services and internships provided by codeclause.com

--------------------------------------------------
tell me about codeclause
1/1 [==============================] - 0s 49ms/step
Code Clause: CodeClause aspires to be the global sourcing choice of the world market and revolutionizes the way service processes function. 
To reach out to the common people across the globe and make Information Technology a tool for the “MASS” along with the tool for the “CLASS”. 
Creating innovative IT solutions and providing IT-enabled services to delight customers worldwide and build Relationships based on Trust, Values and Professionalism. CodeClause has industry-specific software expertise in Technology, Financial, Healthcare, Media, Manufacturing, and many other sectors. 
